In [8]:
from neo4j import GraphDatabase
from rdflib_neo4j import Neo4jStoreConfig, Neo4jStore, HANDLE_VOCAB_URI_STRATEGY
from rdflib import Graph
import perfumeKGQueries as pkg

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://53601604.databases.neo4j.io"
AUTH = ("neo4j", "P2Ui4U8o4mpSotvcLNAVXock_1IpcJh5rFANct7uKSc")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [3]:


def retrieve_and_print_all_accords(driver):
    with driver.session() as session:
        try:
            def get_all_accords(tx):
                query = """
                MATCH (a:Accord)
                RETURN a.uri as uri, a.label as name
                """
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_all_accords)
                print("All Accords:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_with_accord(driver, accordURI):
    with driver.session() as session:
        try:
            def get_perfumes(tx):
                query = pkg.perfumesHaveAccord(accordURI)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_perfumes)
                print("All Perfumes with Searched Accord:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_similar_by_accord(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similar_perfumes(tx):
                query = pkg.perfumesShareAccords(perfume_uri)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['Perfume'], 'score': record['MutualAccords']} for record in result]

            with driver.session() as session:
                accords = session.read_transaction(get_similar_perfumes)
                print("Top 10 Similar Perfumes:")
                for accord in accords:
                        print(f"name: {accord['name']}, uri: {accord['uri']}, score: {accord['score']}")
                        
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_and_print_all_notes(driver):
    with driver.session() as session:
        try:
            def get_all_notes(tx):
                query = """
                MATCH (n:Note)
                RETURN n.uri as uri, n.label as name
                """
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            notes = session.read_transaction(get_all_notes)
            print("All Notes:")
            for note in notes:
                print(f"name: {note['name']}, uri: {note['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_with_note(driver, noteURI):
    with driver.session() as session:
        try:
            def get_perfumes(tx):
                query = pkg.perfumesHaveNote(noteURI)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            perfumes = session.read_transaction(get_perfumes)
            print("All Perfumes with Searched Note:")
            for perfume in perfumes:
                print(f"name: {perfume['name']}, uri: {perfume['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_perfumes_similar_by_note(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similar_perfumes(tx):
                query = pkg.perfumesShareNotes(perfume_uri)

                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['Perfume'], 'score': record['MutualNotes']} for record in result]

            similar_perfumes = session.read_transaction(get_similar_perfumes)
            print("Top 10 Similar Perfumes:")
            for perfume in similar_perfumes:
                print(f"name: {perfume['name']}, uri: {perfume['uri']}, score: {perfume['score']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_and_print_all_brands(driver):
    with driver.session() as session:
        try:
            def get_all_brands(tx):
                query = pkg.getAllBrands()
                result = tx.run(query)
                return [{'uri': record['uri'], 'name': record['name']} for record in result]

            brands = session.read_transaction(get_all_brands)
            print("All Brands:")
            for brand in brands:
                print(f"name: {brand['name']}, uri: {brand['uri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_notes_by_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_notes(tx):
                query = pkg.getNotesUsedByBrand(brand_uri)
                result = tx.run(query)
                notes = []
                for record in result:
                    notes.append({
                        'noteUri': record['noteUri'],
                        'noteName': record['noteName']
                    })
                return notes

            notes = session.read_transaction(get_notes)
            print(f"Notes used by Brand URI: {brand_uri}")
            for note in notes:
                print(f"Note - name: {note['noteName']}, uri: {note['noteUri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_accords_by_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_accords(tx):
                query = pkg.getAccordsUsedByBrand(brand_uri)
                result = tx.run(query)
                accords = []
                for record in result:
                    accords.append({
                        'accordUri': record['accordUri'],
                        'accordName': record['accordName']
                    })
                return accords

            accords = session.read_transaction(get_accords)
            print(f"Accords used by Brand URI: {brand_uri}")
            for accord in accords:
                print(f"Accord - name: {accord['accordName']}, uri: {accord['accordUri']}")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_most_used_accord_for_brand(driver, brand_uri):
    with driver.session() as session:
        try:
            def get_most_used_accord(tx):
                query = pkg.getMostUsedAccordForBrand(brand_uri)
                result = tx.run(query)
                record = result.single()
                if record:
                    return {
                        'brandUri': brand_uri,
                        'accordName': record['accordName'],
                        'accordUri': record['accordUri'],
                        'usageCount': record['usageCount']
                    }
                else:
                    return None

            accord = session.read_transaction(get_most_used_accord)
            if accord:
                print(f"Most Used Accord for Brand URI {brand_uri}:")
                print(f"Accord: {accord['accordName']}, URI: {accord['accordUri']}, Usage Count: {accord['usageCount']}")
            else:
                print(f"No accords found for Brand URI {brand_uri}")
                                
        except Exception as e:
            print(f"An error occurred: {e}")


## Accords

In [3]:
# selected_accord = {'name': 'coffee', 'uri': 'http://www.example.org/perfumeKG#coffee'}
# selected_accord = {'name': 'earthy', 'uri': 'http://www.example.org/perfumeKG#earthy'}
selected_accord = {'name': 'animalic', 'uri': 'http://www.example.org/perfumeKG#animalic'}

In [4]:
retrieve_perfumes_with_accord(driver, selected_accord['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:24: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:32: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:33: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_perfumes)


All Perfumes with Searched Accord:
name: al fareed, uri: http://www.example.org/perfumeKG#alFareed
name: amalfi flowers, uri: http://www.example.org/perfumeKG#amalfiFlowers
name: amber oud ultra violet, uri: http://www.example.org/perfumeKG#amberOudUltraViolet
name: amber wood, uri: http://www.example.org/perfumeKG#amberWood
name: ambilux, uri: http://www.example.org/perfumeKG#ambilux
name: aqua allegoria nettare di sole, uri: http://www.example.org/perfumeKG#aquaAllegoriaNettareDiSole
name: atkinsons his majesty the oud, uri: http://www.example.org/perfumeKG#atkinsonsHisMajestyTheOud
name: aura loewe magnética, uri: http://www.example.org/perfumeKG#auraLoeweMagnética
name: authentic night femme, uri: http://www.example.org/perfumeKG#authenticNightFemme
name: beauty-to-wear bohemian belle, uri: http://www.example.org/perfumeKG#beauty-to-wearBohemianBelle
name: black datura, uri: http://www.example.org/perfumeKG#blackDatura
name: boss bottled parfum, uri: http://www.example.org/perfumeK

In [5]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#fairytale'}

In [6]:
retrieve_perfumes_similar_by_accord(driver, selected_perfume['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:43: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:51: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1877246278.py:52: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_similar_perfumes)


Top 10 Similar Perfumes:
name: galloway, uri: http://www.example.org/perfumeKG#galloway, score: 7
name: classique la robe chinoise, uri: http://www.example.org/perfumeKG#classiqueLaRobeChinoise, score: 7
name: dance, uri: http://www.example.org/perfumeKG#dance, score: 7
name: exult, uri: http://www.example.org/perfumeKG#exult, score: 7
name: aura mugler, uri: http://www.example.org/perfumeKG#auraMugler, score: 7
name: cherry of cashmere, uri: http://www.example.org/perfumeKG#cherryOfCashmere, score: 7
name: by invitation, uri: http://www.example.org/perfumeKG#byInvitation, score: 7
name: tuberose, uri: http://www.example.org/perfumeKG#tuberose, score: 7
name: esoteric, uri: http://www.example.org/perfumeKG#esoteric, score: 7
name: happy spirit, uri: http://www.example.org/perfumeKG#happySpirit, score: 7


## Notes

In [8]:
retrieve_and_print_all_notes(driver)

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:61: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:71: DeprecationWarning: read_transaction has been renamed to execute_read
  notes = session.read_transaction(get_all_notes)


All Notes:
name: None, uri: http://www.example.org/perfumeKG#MuskNote
name: None, uri: http://www.example.org/perfumeKG#LeatherNote
name: None, uri: http://www.example.org/perfumeKG#SandalwoodNote
name: orris root, uri: http://www.example.org/perfumeKG#orrisRootNote
name: white musk, uri: http://www.example.org/perfumeKG#whiteMuskNote
name: patchouli, uri: http://www.example.org/perfumeKG#patchouliNote
name: vanilla, uri: http://www.example.org/perfumeKG#vanillaNote
name: cedar, uri: http://www.example.org/perfumeKG#cedarNote
name: tonka bean, uri: http://www.example.org/perfumeKG#tonkaBeanNote
name: styrax, uri: http://www.example.org/perfumeKG#styraxNote
name: sandalwood, uri: http://www.example.org/perfumeKG#sandalwoodNote
name: musk, uri: http://www.example.org/perfumeKG#muskNote
name: castoreum, uri: http://www.example.org/perfumeKG#castoreumNote
name: tobacco, uri: http://www.example.org/perfumeKG#tobaccoNote
name: agarwood (oud), uri: http://www.example.org/perfumeKG#agarwood(ou

In [9]:
selected_note = {'name': 'vetiver', 'uri': 'http://www.example.org/perfumeKG#vetiverNote'}

In [10]:
retrieve_perfumes_with_note(driver, selected_note['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:80: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:88: DeprecationWarning: read_transaction has been renamed to execute_read
  perfumes = session.read_transaction(get_perfumes)


All Perfumes with Searched Note:
name: amber wood, uri: http://www.example.org/perfumeKG#amberWood
name: amore, uri: http://www.example.org/perfumeKG#amore
name: aromatics elixir sheer velvet philtre sensuel, uri: http://www.example.org/perfumeKG#aromaticsElixirSheerVelvetPhiltreSensuel
name: beyoncé heat the mrs. carter show world tour limited edition, uri: http://www.example.org/perfumeKG#beyoncéHeatTheMrs.CarterShowWorldTourLimitedEdition
name: blue land, uri: http://www.example.org/perfumeKG#blueLand
name: bogart story green, uri: http://www.example.org/perfumeKG#bogartStoryGreen
name: boss bastard, uri: http://www.example.org/perfumeKG#bossBastard
name: boss bottled united, uri: http://www.example.org/perfumeKG#bossBottledUnited
name: cabotine bleu, uri: http://www.example.org/perfumeKG#cabotineBleu
name: capri, uri: http://www.example.org/perfumeKG#capri
name: cow, uri: http://www.example.org/perfumeKG#cow
name: dolce di giorno, uri: http://www.example.org/perfumeKG#dolceDiGiorno

In [11]:
selected_perfume_by_note = {'name': 'grace', 'uri': 'http://www.example.org/perfumeKG#grace'}

In [12]:
retrieve_perfumes_similar_by_note(driver, selected_perfume_by_note['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:97: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_5508\1291535366.py:105: DeprecationWarning: read_transaction has been renamed to execute_read
  similar_perfumes = session.read_transaction(get_similar_perfumes)


Top 10 Similar Perfumes:
name: moon light, uri: http://www.example.org/perfumeKG#moonLight, score: 6
name: irth, uri: http://www.example.org/perfumeKG#irth, score: 6
name: antidote, uri: http://www.example.org/perfumeKG#antidote, score: 5
name: trussardi inside for women, uri: http://www.example.org/perfumeKG#trussardiInsideForWomen, score: 4
name: dolce di giorno, uri: http://www.example.org/perfumeKG#dolceDiGiorno, score: 4
name: uomo - legno marino, uri: http://www.example.org/perfumeKG#uomo-LegnoMarino, score: 4
name: guess 1981 los angeles men, uri: http://www.example.org/perfumeKG#guess1981LosAngelesMen, score: 4
name: golden sand, uri: http://www.example.org/perfumeKG#goldenSand, score: 4
name: boss bottled united, uri: http://www.example.org/perfumeKG#bossBottledUnited, score: 3
name: 1957 eau de parfum, uri: http://www.example.org/perfumeKG#1957EauDeParfum, score: 3


## Brands

In [3]:
retrieve_and_print_all_brands(driver)

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:113: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:120: DeprecationWarning: read_transaction has been renamed to execute_read
  brands = session.read_transaction(get_all_brands)


All Brands:
name: None, uri: http://www.example.org/perfumeKG#TUMI
name: ulrich lang, uri: http://www.example.org/perfumeKG#Brand_ulrichLang
name: histoires de parfums, uri: http://www.example.org/perfumeKG#Brand_histoiresDeParfums
name: chanel, uri: http://www.example.org/perfumeKG#Brand_chanel
name: rosendo mateu olfactive expressions, uri: http://www.example.org/perfumeKG#Brand_rosendoMateuOlfactiveExpressions
name: paco rabanne, uri: http://www.example.org/perfumeKG#Brand_pacoRabanne
name: yves saint laurent, uri: http://www.example.org/perfumeKG#Brand_yvesSaintLaurent
name: afnan, uri: http://www.example.org/perfumeKG#Brand_afnan
name: giorgio armani, uri: http://www.example.org/perfumeKG#Brand_giorgioArmani
name: bath & body works, uri: http://www.example.org/perfumeKG#Brand_bath&BodyWorks
name: yves rocher, uri: http://www.example.org/perfumeKG#Brand_yvesRocher
name: i profumi di firenze, uri: http://www.example.org/perfumeKG#Brand_iProfumiDiFirenze
name: the different company, 

In [3]:
selected_brand = {'name': 'zara', 'uri': 'http://www.example.org/perfumeKG#Brand_zara'}

In [5]:
print(pkg.getAccordsUsedByBrand(selected_brand['uri']))


        MATCH (brand:Brand {uri: 'http://www.example.org/perfumeKG#Brand_zara'})<-[:producedBy]-(perfume:Perfume)-[:hasAccord]->(accord:Accord)
        RETURN DISTINCT accord.uri AS accordUri, accord.label AS accordName
    


In [6]:
retrieve_notes_by_brand(driver, selected_brand['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:129: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:142: DeprecationWarning: read_transaction has been renamed to execute_read
  notes = session.read_transaction(get_notes)


Notes used by Brand URI: http://www.example.org/perfumeKG#Brand_zara
Note - name: orange blossom, uri: http://www.example.org/perfumeKG#Note_orangeBlossom
Note - name: bergamot, uri: http://www.example.org/perfumeKG#Note_bergamot
Note - name: mandarin orange, uri: http://www.example.org/perfumeKG#Note_mandarinOrange
Note - name: amber, uri: http://www.example.org/perfumeKG#Note_amber
Note - name: vanilla, uri: http://www.example.org/perfumeKG#Note_vanilla
Note - name: honey, uri: http://www.example.org/perfumeKG#Note_honey
Note - name: tobacco, uri: http://www.example.org/perfumeKG#Note_tobacco
Note - name: praline, uri: http://www.example.org/perfumeKG#Note_praline
Note - name: apple, uri: http://www.example.org/perfumeKG#Note_apple
Note - name: coconut water, uri: http://www.example.org/perfumeKG#Note_coconutWater
Note - name: rum, uri: http://www.example.org/perfumeKG#Note_rum
Note - name: guaiac wood, uri: http://www.example.org/perfumeKG#Note_guaiacWood
Note - name: black pepper, 

In [7]:
retrieve_accords_by_brand(driver, selected_brand['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:151: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_15600\3992507684.py:164: DeprecationWarning: read_transaction has been renamed to execute_read
  accords = session.read_transaction(get_accords)


Accords used by Brand URI: http://www.example.org/perfumeKG#Brand_zara
Accord - name: Accord_citrus, uri: http://www.example.org/perfumeKG#Accord_citrus
Accord - name: Accord_whiteFloral, uri: http://www.example.org/perfumeKG#Accord_whiteFloral
Accord - name: Accord_fruity, uri: http://www.example.org/perfumeKG#Accord_fruity
Accord - name: Accord_powdery, uri: http://www.example.org/perfumeKG#Accord_powdery
Accord - name: Accord_amber, uri: http://www.example.org/perfumeKG#Accord_amber
Accord - name: Accord_fresh, uri: http://www.example.org/perfumeKG#Accord_fresh
Accord - name: Accord_sweet, uri: http://www.example.org/perfumeKG#Accord_sweet
Accord - name: Accord_vanilla, uri: http://www.example.org/perfumeKG#Accord_vanilla
Accord - name: Accord_tobacco, uri: http://www.example.org/perfumeKG#Accord_tobacco
Accord - name: Accord_honey, uri: http://www.example.org/perfumeKG#Accord_honey
Accord - name: Accord_rum, uri: http://www.example.org/perfumeKG#Accord_rum
Accord - name: Accord_woo

In [8]:
print(pkg.getMostUsedAccordForBrand(selected_brand['uri']))


        MATCH (brand:Brand {uri: 'http://www.example.org/perfumeKG#Brand_zara'})<-[:producedBy]-(perfume:Perfume)-[:hasAccord]->(accord:Accord)
        WITH accord, COUNT(*) AS usageCount
        ORDER BY usageCount DESC
        RETURN accord.label AS accordName, accord.uri AS accordUri, usageCount
        LIMIT 1
    


In [7]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#Brand_royalCrown'}

In [5]:
pkg.get_most_similar_perfume_query(selected_perfume['uri'])

TypeError: get_most_similar_perfume_query() takes 0 positional arguments but 1 was given

In [6]:
print(pkg.perfumesShareAccords(selected_perfume['uri']))


        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#fairytale'})-[:hasAccord]->(accord:Accord)<-[:hasAccord]-(other:Perfume)
        WHERE other <> specific
        RETURN other.label AS Perfume, other.uri AS uri, COUNT(accord) AS MutualAccords
        ORDER BY MutualAccords DESC
        LIMIT 10
    


## Perfume Similarity

In [10]:
selected_perfume = {'name': 'fairytale', 'uri': 'http://www.example.org/perfumeKG#Perfume_fairytale'}

print(pkg.getAccordSimilarityScore(selected_perfume['uri']))


        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#Perfume_fairytale'})
        OPTIONAL MATCH (specific)-[:hasAccord]->(specificAccord:Accord)
        WITH specific, COLLECT(DISTINCT specificAccord) AS specificAccords

        MATCH (other:Perfume)
        WHERE other <> specific
        OPTIONAL MATCH (other)-[:hasAccord]->(otherAccord:Accord)
        WITH specific, specificAccords, other, COLLECT(DISTINCT otherAccord) AS otherAccords

        WITH other,
            [accord IN specificAccords WHERE accord IN otherAccords] AS mutualAccords,
            [accord IN specificAccords WHERE NOT accord IN otherAccords] +
            [accord IN otherAccords WHERE NOT accord IN specificAccords] AS notMutualAccords,
            SIZE([accord IN specificAccords WHERE accord IN otherAccords]) AS MutualAccordsCount,
            SIZE([accord IN specificAccords WHERE NOT accord IN otherAccords] +
                [accord IN otherAccords WHERE NOT accord IN specificAccords]) AS N

In [25]:
def retrieve_accord_similarity_scores(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similarity_info(tx):
                query = pkg.getAccordSimilarityScore(perfume_uri)
                result = tx.run(query)
                return [{
                    'Perfume': record['Perfume'],
                    'uri': record['uri'],
                    'MutualAccordsCount': record['MutualAccordsCount'],
                    'NotMutualAccordsCount': record['NotMutualAccordsCount'],
                    'similarity': record['similarity']
                } for record in result]

            perfumes_info = session.read_transaction(get_similarity_info)
            print(f"Similarity scores compared with '{perfume_uri}':")
            for info in perfumes_info:
                print(f"Name: {info['Perfume']}")
                print(f"URI: {info['uri']}")
                print(f"Mutual Accords Count: {info['MutualAccordsCount']}")
                print(f"Not Mutual Accords Count: {info['NotMutualAccordsCount']}")
                print(f"Similarity Score: {info['similarity']}")
                print("---")
                            
        except Exception as e:
            print(f"An error occurred: {e}")

def retrieve_note_similarity_scores(driver, perfume_uri):
    with driver.session() as session:
        try:
            def get_similarity_info(tx):
                query = pkg.getNoteSimilarityScore(perfume_uri)
                result = tx.run(query)
                return [{
                    'Perfume': record['Perfume'],
                    'uri': record['uri'],
                    'MutualNotesCount': record['MutualNotesCount'],
                    'NotMutualNotesCount': record['NotMutualNotesCount'],
                    'similarity': record['similarity']
                } for record in result]

            perfumes_info = session.read_transaction(get_similarity_info)
            print(f"Similarity scores based on notes compared with '{perfume_uri}':")
            for info in perfumes_info:
                print(f"Name: {info['Perfume']}")
                print(f"URI: {info['uri']}")
                print(f"Mutual Notes Count: {info['MutualNotesCount']}")
                print(f"Not Mutual Notes Count: {info['NotMutualNotesCount']}")
                print(f"Similarity Score: {info['similarity']}")
                print("---")
                                
        except Exception as e:
            print(f"An error occurred: {e}")

In [30]:
def retrieve_combined_similarity_scores(driver, perfume_uri):
    accord_similarities = retrieve_accord_similarity_scores(driver, perfume_uri)
    note_similarities = retrieve_note_similarity_scores(driver, perfume_uri)
    
    combined_scores = {}

    # Process accord similarities
    if accord_similarities:
        for accord_info in accord_similarities:
            uri = accord_info['uri']
            accord_similarity = accord_info.get('similarity') or 0  # Set to 0 if None
            combined_scores[uri] = {
                'Perfume': accord_info['Perfume'],
                'uri': uri,
                'accordSimilarity': accord_similarity,
                'noteSimilarity': 0  # Default to 0
            }

    # Process note similarities
    if note_similarities:
        for note_info in note_similarities:
            uri = note_info['uri']
            note_similarity = note_info.get('similarity') or 0  # Set to 0 if None
            if uri in combined_scores:
                combined_scores[uri]['noteSimilarity'] = note_similarity
            else:
                combined_scores[uri] = {
                    'Perfume': note_info['Perfume'],
                    'uri': uri,
                    'accordSimilarity': 0,
                    'noteSimilarity': note_similarity
                }

    # Handle perfumes that are not in accord_similarities or note_similarities
    # If both accord_similarities and note_similarities are empty, combined_scores will be empty

    # Calculate averageSimilarity
    for perfume in combined_scores.values():
        perfume['averageSimilarity'] = (perfume['accordSimilarity'] + perfume['noteSimilarity']) / 2.0

    # Sort the perfumes by averageSimilarity in descending order and take top 10
    top_perfumes = sorted(combined_scores.values(), key=lambda x: x['averageSimilarity'], reverse=True)[:10]

    print("Top 10 Combined Similarity Scores:")
    for perfume in top_perfumes:
        print(f"Name: {perfume['Perfume']}")
        print(f"URI: {perfume['uri']}")
        print(f"Accord Similarity: {perfume['accordSimilarity']}")
        print(f"Note Similarity: {perfume['noteSimilarity']}")
        print(f"Average Similarity: {perfume['averageSimilarity']}")
        print("---")


In [31]:
retrieve_combined_similarity_scores(driver, selected_perfume['uri'])

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_2856\2083209521.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_2856\2083209521.py:15: DeprecationWarning: read_transaction has been renamed to execute_read
  perfumes_info = session.read_transaction(get_similarity_info)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n        MATCH (specific:Perfume {uri: 'http://www.example.org/perfumeKG#Perfume_fairytale'})\n        OPTIONAL MATCH (specific)-[:hasAccord]->(specificAccord:Accord)\n        WITH specific, COLLECT(DISTINCT specificAccord) AS specificAccords\n\n      

Similarity scores compared with 'http://www.example.org/perfumeKG#Perfume_fairytale':
Name: sweet pulp
URI: http://www.example.org/perfumeKG#Perfume_sweetPulp
Mutual Accords Count: 7
Not Mutual Accords Count: 3
Similarity Score: 0.6363636363636364
---
Name: aura mugler
URI: http://www.example.org/perfumeKG#Perfume_auraMugler
Mutual Accords Count: 7
Not Mutual Accords Count: 5
Similarity Score: 0.5384615384615384
---
Name: happy spirit
URI: http://www.example.org/perfumeKG#Perfume_happySpirit
Mutual Accords Count: 7
Not Mutual Accords Count: 5
Similarity Score: 0.5384615384615384
---
Name: by invitation
URI: http://www.example.org/perfumeKG#Perfume_byInvitation
Mutual Accords Count: 7
Not Mutual Accords Count: 6
Similarity Score: 0.5
---
Name: cherry of cashmere
URI: http://www.example.org/perfumeKG#Perfume_cherryOfCashmere
Mutual Accords Count: 7
Not Mutual Accords Count: 6
Similarity Score: 0.5
---
Name: classique la robe chinoise
URI: http://www.example.org/perfumeKG#Perfume_classiqu